# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame

cities_df = pd.read_csv('../output_data/cities.csv')
cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Soyo,-6.1349,12.3689,297.89,81,93,2.14,AO,1635364128
1,1,Ribeira Grande,38.5167,-28.7000,293.37,73,20,7.20,PT,1635364128
2,2,Provideniya,64.3833,-173.3000,266.21,63,75,5.00,RU,1635364128
3,3,Hvolsvollur,63.7533,-20.2243,276.38,77,99,2.54,IS,1635364129
4,4,Vila Franca do Campo,37.7167,-25.4333,292.38,90,100,4.05,PT,1635364129
...,...,...,...,...,...,...,...,...,...,...
590,590,Sechura,-5.5569,-80.8222,298.07,58,72,7.30,PE,1635364316
591,591,Neuquén,-38.9516,-68.0591,303.09,20,0,3.60,AR,1635364062
592,592,Bonthe,7.5264,-12.5050,300.42,79,69,4.72,SL,1635364316
593,593,Tezu,27.9167,96.1667,292.23,52,1,1.95,IN,1635364316


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Use the Lat and Lng as locations and Humidity as the weight.

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity'].astype(float)

## Add Heatmap layer to map.

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create new DataFrame fitting weather criteria

# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

narrCities_df = cities_df.loc[(cities_df["Max Temp"] > 294) & (cities_df["Max Temp"] < 299)  & 
                                 (cities_df["Wind Speed"] < 10)& 
                                 (cities_df["Cloudiness"] == 0)]

narrCities_df = narrCities_df.dropna(how='any')
narrCities_df.reset_index(inplace=True)
del narrCities_df['index']
narrCities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,21,Chui,-33.6971,-53.4616,298.41,57,0,3.10,UY,1635364134
1,70,Pisco,-13.7000,-76.2167,294.18,64,0,8.23,PE,1635364146
2,78,Kiryat Gat,31.6100,34.7642,294.71,63,0,2.78,IL,1635363903
3,96,Cidreira,-30.1811,-50.2056,295.95,68,0,4.21,BR,1635364154
4,115,Mar del Plata,-38.0023,-57.5575,298.16,87,0,4.92,AR,1635363950
5,136,Saint-Pierre,-21.3393,55.4781,294.97,71,0,3.09,RE,1635364166
6,190,San Quintín,30.4833,-115.9500,295.62,45,0,3.19,MX,1635364119
7,213,Bariloche,-41.1456,-71.3082,296.73,21,0,9.77,AR,1635363930
8,313,Calama,-22.4667,-68.9333,296.42,9,0,9.77,CL,1635364225
9,320,Ormara,25.2088,64.6357,298.61,65,0,3.30,PK,1635364226


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.

hotel_df = narrCities_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,Chui,-33.6971,-53.4616,298.41,57,0,3.10,UY,1635364134,
1,70,Pisco,-13.7000,-76.2167,294.18,64,0,8.23,PE,1635364146,
2,78,Kiryat Gat,31.6100,34.7642,294.71,63,0,2.78,IL,1635363903,
3,96,Cidreira,-30.1811,-50.2056,295.95,68,0,4.21,BR,1635364154,
4,115,Mar del Plata,-38.0023,-57.5575,298.16,87,0,4.92,AR,1635363950,
5,136,Saint-Pierre,-21.3393,55.4781,294.97,71,0,3.09,RE,1635364166,
6,190,San Quintín,30.4833,-115.9500,295.62,45,0,3.19,MX,1635364119,
7,213,Bariloche,-41.1456,-71.3082,296.73,21,0,9.77,AR,1635363930,
8,313,Calama,-22.4667,-68.9333,296.42,9,0,9.77,CL,1635364225,
9,320,Ormara,25.2088,64.6357,298.61,65,0,3.30,PK,1635364226,


In [6]:
for city in hotel_df['City'].array:
    print(hotel_df.loc[(hotel_df['City'] == city)]['Lat'].values[0])

-33.6971
-13.7
31.61
-30.1811
-38.0023
-21.3393
30.4833
-41.1456
-22.4667
25.2088
36.2083
17.0151
28.9684
32.3754
34.8986
26.2958
34.6959


In [7]:
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

hotel_names = []

for city in hotel_df['City'].array:
    lat = hotel_df.loc[(hotel_df['City'] == city)]['Lat'].values[0]
    lng = hotel_df.loc[(hotel_df['City'] == city)]['Lng'].values[0]
    target_search = 'hotel'
    target_rad = 5000
    target_type = 'Hotel'
    
    params = {
        "location": f'{lat}, {lng}',
        "keyword": target_search,
        "radius": target_rad,
        "type": target_type,
        "key": g_key
    }
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    try:
        # run a request using our params dictionary
        response = requests.get(base_url, params=params).json()
        hotel_names.append(response['results'][0]['name'])
        
    except (IndexError):
        print(f'{city} failed')
        hotel_names.append(np.nan)
        continue
hotel_df['Hotel Name'] = hotel_names
hotel_df

Jām Sāhib failed


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,21,Chui,-33.6971,-53.4616,298.41,57,0,3.10,UY,1635364134,Bertelli Chuí Hotel
1,70,Pisco,-13.7000,-76.2167,294.18,64,0,8.23,PE,1635364146,Hotel San Jorge Residencial
2,78,Kiryat Gat,31.6100,34.7642,294.71,63,0,2.78,IL,1635363903,Desert Gat
3,96,Cidreira,-30.1811,-50.2056,295.95,68,0,4.21,BR,1635364154,Hotel Castelo
4,115,Mar del Plata,-38.0023,-57.5575,298.16,87,0,4.92,AR,1635363950,Hotel Sainte Jeanne
5,136,Saint-Pierre,-21.3393,55.4781,294.97,71,0,3.09,RE,1635364166,Hôtel Le Saint-Pierre
6,190,San Quintín,30.4833,-115.9500,295.62,45,0,3.19,MX,1635364119,Old Mill Hotel
7,213,Bariloche,-41.1456,-71.3082,296.73,21,0,9.77,AR,1635363930,Hotel NH Bariloche Edelweiss
8,313,Calama,-22.4667,-68.9333,296.42,9,0,9.77,CL,1635364225,ibis budget Calama
9,320,Ormara,25.2088,64.6357,298.61,65,0,3.30,PK,1635364226,Ormara Beach Resorts/ Bolan Safari


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))